In [1]:
import numpy as np
import h5py
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file = h5py.File('/content/drive/MyDrive/BCI/MI_3.mat', 'r')
X = np.array(file['X'])  # (576, 3, 500)
X = X.transpose(2, 1, 0)  # → (576, 500, 3)
X = X.reshape(-1, 500)  # (576 × 3, 500) → (1728, 500)

y = np.array(file['y']).flatten()  # (576,)
# 標籤也要複製三次，讓每個 channel 對應正確 label
y = np.repeat(y, 3)  # 576 → 1728
print(f'X shape: {X.shape}, y shape: {y.shape}')

# === 分割為 training-validation & testing ===
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# === Cross-validation 設定 ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_acc_list = []
val_acc_list = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

fold = 1
for train_idx, val_idx in skf.split(X_trainval, y_trainval):
    X_train, X_val = X_trainval[train_idx], X_trainval[val_idx]
    y_train, y_val = y_trainval[train_idx], y_trainval[val_idx]

    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # 訓練 KNN 模型（K 可自行調整）
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_scaled, y_train)

    # 預測
    y_train_pred = knn.predict(X_train_scaled)
    y_val_pred = knn.predict(X_val_scaled)

    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)

    print(f'📘 Fold {fold}: Train Acc = {train_acc:.4f} | Val Acc = {val_acc:.4f}')
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    fold += 1

    # 每類別正確率統計
    for true, pred in zip(y_val, y_val_pred):
        class_total[true] += 1
        if true == pred:
            class_correct[true] += 1

# === 使用全部訓練資料重新訓練並測試 ===
scaler_final = StandardScaler()
X_trainval_scaled = scaler_final.fit_transform(X_trainval)
X_test_scaled = scaler_final.transform(X_test)

knn_final = KNeighborsClassifier(n_neighbors=5)
knn_final.fit(X_trainval_scaled, y_trainval)

y_test_pred = knn_final.predict(X_test_scaled)
test_acc = accuracy_score(y_test, y_test_pred)

# === 顯示結果 ===
print("\n✅ Cross-Validation Summary")
print(f'📊 平均 Train Acc: {np.mean(train_acc_list):.4f}')
print(f'🧪 平均 Val Acc: {np.mean(val_acc_list):.4f}')

print("\n🎯 每類別驗證集正確率：")
label_names = ['Left Hand', 'Right Hand']
for i in [0, 1]:
    acc = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print(f'📡 {label_names[i]} Acc: {acc:.4f} ({class_correct[i]}/{class_total[i]})')

print(f'\n🚀 測試集準確率: {test_acc:.4f}')
print("\n📋 測試集分類報告:")
print(classification_report(y_test, y_test_pred, target_names=label_names))

X shape: (1728, 500), y shape: (1728,)
📘 Fold 1: Train Acc = 0.8724 | Val Acc = 0.6679
📘 Fold 2: Train Acc = 0.8543 | Val Acc = 0.7148
📘 Fold 3: Train Acc = 0.8716 | Val Acc = 0.6920
📘 Fold 4: Train Acc = 0.8626 | Val Acc = 0.6703
📘 Fold 5: Train Acc = 0.8725 | Val Acc = 0.6884

✅ Cross-Validation Summary
📊 平均 Train Acc: 0.8667
🧪 平均 Val Acc: 0.6867

🎯 每類別驗證集正確率：
📡 Left Hand Acc: 0.5456 (377/691)
📡 Right Hand Acc: 0.8278 (572/691)

🚀 測試集準確率: 0.7399

📋 測試集分類報告:
              precision    recall  f1-score   support

   Left Hand       0.81      0.62      0.71       173
  Right Hand       0.69      0.86      0.77       173

    accuracy                           0.74       346
   macro avg       0.75      0.74      0.74       346
weighted avg       0.75      0.74      0.74       346

